## Import

In [1]:
import os
from dotenv import load_dotenv

# import prompt template
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# import the json oupput parser from the langchain core
from langchain_core.output_parsers import JsonOutputParser

d:\FSA\fsa-simpleqt\backend-simpleqt\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.memory import ConversationBufferMemory

In [3]:
from langchain.chains import LLMChain

In [4]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
import pickle

In [6]:
# define the parser object
parser = JsonOutputParser()

# Import API key
load_dotenv()

# Define the google api key
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['CLAUDE_API_KEY'] = os.getenv('CLAUDE_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
CLAUDE_API_KEY = os.environ.get("CLAUDE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

Python-dotenv could not parse statement starting at line 1


## Model, Prompts,...

In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, convert_system_message_to_human=True, api_key=GOOGLE_API_KEY, request_timeout=120)

In [8]:
# Summarize JD
prompt_jd = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """
                Let's think step by step.
                Act as a HR manager and analyze Job Description: {jd}
                Analyzing guide:
                Let's think step by step.
                Respond using only the provided information and do not rely on your basic knowledge.
                Only use the given data to determine educational qualifications and certificates; do not make assumptions about these qualifications.
                However, you are allowed to combine the provided details to draw logical conclusions about soft skills.
                Please structure the JSON output as follows: {"Job Description": "", "Requirements": ""}
                Next, you will be given CVs. Analyze the CVs and determine if the candidates meet the job requirements. Be fair and objective in your assessment.
                """
            )
        ),
        HumanMessagePromptTemplate.from_template(["{jd}"]),
    ]
)

In [9]:
# Match CV to JD
prompt_match = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """
                I have a curriculum vitae (CV) for a candidate, and the above job description (JD) for a specific position. 
                This is the CV: 
                {cv}
                Based on the provided CV and JD, I need an analysis that examines how well the candidate's qualifications match the job requirements. This analysis should be summarized in a JSON file, with special attention to:
                - The match with the base requirements outlined in the JD, including education, technical skills, and relevant experiences.
                - A detailed description of any additional qualifications the candidate possesses that align with the JD. This should focus on research experience, communication skills, and any other notable achievements, directly inserted into the 'bonus_qualifications_met' field in the JSON.
                - An estimated match percentage reflecting how closely the candidate fits the job description, based on the analysis.
                - The total count of bonus qualifications identified in the candidate's background.

                Please structure the JSON output as follows:

                ```json
                {
                "match_with_base_requirements": "Include a detailed analysis here, linking the candidate's qualifications directly to the job's base requirements.",
                "bonus_qualifications_met": "Insert detailed descriptions of the candidate's bonus qualifications here, emphasizing areas such as research experience, communication skills, and other relevant achievements.",
                "candidate_match_percentage": "Provide an estimated match percentage here based on the analysis.",
                "number_of_bonus_qualifications": "Indicate the total number of detailed bonus qualifications identified."
                }
                """
            )
        ),
        HumanMessagePromptTemplate.from_template(["{cv}"]),
    ]
)

In [10]:
jd = """
Job Description for AI Engineer Fresher at GMO-Z.com RUNSYSTEM
1) Job Description:
Research and develop new AI solutions and technologies for the company's software projects using artificial intelligence technology.
Participate in knowledge sharing seminars and regular reports.
Undertake other projects and tasks as required by the direct manager.
2) Requirements:
Knowledge and skills:
Graduate from a regular university majoring in Computer Science, Data Science, or Artificial Intelligence.
Proficiency in one of the programming languages such as Python, C++, etc.
Experience working with Git, Docker.
Understand the frameworks: Tensorflow/keras, pytorch.
Understand the architectures CNN, RNN, LSTM, Transformer.
Understand the SSH network protocol.
Other requirements:
Read and understand English in the field of Information Technology.
Be interested in and passionate about researching problems related to face processing systems.
Have a spirit of hard work, responsibility for assigned tasks and long-term commitment to the company.
Good communication skills, ability to work independently and in a team.
Experience: Experience in participating in AI projects related to face processing systems is an advantage.
Priority:
Candidates who have researched AI/Machine Learning topics in laboratories or have international publications and scientific awards are an advantage.
Good communication English.
3) Benefits:
Salary: Up to 10,000,000 VND/month.
Performance bonus, Project bonus and other bonuses (Achievement bonus, Outstanding individual bonus, Foreign language/Professional certificate bonus).
Work directly with Japanese clients. Opportunity to Onsite in Japan (when becoming a full-time employee).
Enjoy other benefits such as: Monthly team building, annual vacation, periodic health check-up...
All regimes in accordance with the Labor Law of Vietnam (Social Insurance, Health Insurance, Unemployment Insurance, ...).
Annual leave and other holidays according to the regulations of the Government of Vietnam.
Young and dynamic working environment.
"""

In [11]:
cv1 = """
DAO NGOC HUY FPT University AI Intern\n\n(cid:17) 28-02-2002\n\n(cid:211) 0913596890\n\ng Huy Dao\n\n(cid:135) Dao Ngoc Huy\n\ndnhuy2802@gmail.com\n\nDao Ngoc Huy\n\nMAIN OBJECT\n\nSTRENGTHS\n\nUnderstand the process of an AI project in the company.\n\nProblem-Solving: Adept at analyzing complex problems.\n\nParticipate in programming support for AI projects or join new projects.\n\nCommunication: Eloquent, good communication and teamwork.\n\nWork as a fresher at the company after internship.\n\nSelf-learing: Discover and practice new knowledge yourself.\n\nEXPERIENCE\n\n2023\n\nZalo AI Challenge 2023\n\nCompetition\n\nJoin track Advertising Banner Generation • Approach the problem in two directions: Using Diffusion Stable and using Prompt Engineering\n\n2023\n\nFPT AI Challenge 2023\n\nCompetition\n\nLearning NLP, build Transformer from scratch using Python. • Make idea and apply fine-tuning model on Huggingface.\n\n2023\n\nFPT Digital Race 2023\n\nCompetition\n\nBe the team leader and achieve 3rd place overall. • Data-centric AI: Using Augmentation data methods for images collected from self-driving vehicles. • Handle imbalanced data before training the model.\n\n2023\n\nResearch: Comparing different methods on imputation task of Time series\n\nUniversity • Apply Machine Learning methods: Linear Regression, KNN, SVM, Decision Tree, Random Forest, Ada\n\nBoost, ...\n\nApply Deep Learing methods: Simple RNN, LSTM, CNN. • ML methods are built from Sklearn, DL methods are built from Keras.\n\n2021 - 2022\n\nTech Lead GDSC FPTU Da Nang Club\n\nUniversity\n\nManage and coordinate work for members. • Collaborate, make content and work with speaker in technology events, webinars, and workshops. • Learning points: Assign appropriate work to members, cooperate with other teams, manage work progress and make timeline technology event.\n\nKNOWLEDGE Exploratory Data Analysis (EDA) - Using Pandas, Matplotlib, Seaborn to find insight into data and preprocessing table data.\n\nSKILLS Programing Languages: SQL, Python, JavaScript.\n\nMachine Learning (ML) - Have knowledge about: Linear Regression, KNN, SVN, Random Forest, Deci- sion Tree, Boosting Algorithm, Stasking,... - Using ML Algorithm for different tasks: classifying, problems related to predic- tion, filling missing data, and other problems.\n\nLibraries: Flask, Pandas, Matplotlib, Plotly, Scikit-learn, librosa, OpenCV...\n\nAI Frameworks: TensorFlow/Keras, PyTorch\n\nDeep Learning (DL) - CNN Architecture: Have knowledge of different CNN models. Build DenseNet and ResNet using Pytorch. - Sequence Model: RNN, LSTM, Attention, Transformer.\n\nTool: Docker, Weight&Bias, Tensorboard, Postman, Colab, ...\n\nExtract Feature - Image: Hough Transform, Edge Detector, SIFT,... - Audio: Spectrogram, RMS-E, Chroma, MFCC, HPSS ...\n\nBackend - Using Flask to write API. - Usìng a database such as MongoDB, or Firebase.
"""

In [12]:
cv2 = """
Đoàn Quang Minh\n\nAI Intern\n\nquangminh57dng@gmail.com\n\n+84 905 070 942\n\n06 Đỗ Thế Chấp, Đà Nẵng\n\ngithub.com/Ming-doan\n\nlinkedin.com/in/quangminhdoan\n\nExperiments 06/2023 Present\n\nTime Series Conference Paper AI Developer\n\n- - - -\n\nExperiment in Time Series or Signal processing Building Training pipeline with Python via Colab Enhance Tuning parameters with over 2,000 runs in 8 hours Experiment in Machine Learning methods with Scikit-learn Experiment in Managing experiment results\n\n10/2023\n\nQuy Nhon AI Hackathon 2023 AI Developer\n\nGet notices by mentors - -\n\nExperiment in building and training Transformer models Experiment in building Recommendation system\n\n09/2023\n\nFPT Digital Race Competition AI Developer\n\nWinning Third prize - -\n\nBuild training & Data augmentation pipeline with Python Enhance Development cycle in AI process\n\n07/2023\n\nUnihack 2023 AI Developer & Full-stack developer\n\nWinning Second prize - - -\n\nExperiment in Object detection with YOLOv8 and TACO dataset Experiment in building AI Back-end system with Flask Experiment in building Mobile with Flutter\n\n05/2023 07/2023\n\nFace Recognition AI Developer & Full-stack developer\n\n- -\n\nExperiment in building Face Recognition solution Experiment in building AI Back-en system with Flask Experiment in building Web with ReactJS\n\n04/2023 Present\n\nEnhance Face Recognition with Ensemble Deep Learning AI Developer & AI Researcher\n\n-\n\nExperiment in building Face Recognition with SOTA models Experiment in writing scientific paper\n\nCertificates\n\nDeep Learning AI AI Deep Learning course by Andrew Ng.\n\n12/2023\n\nGDG Devfest Devfest Hackathon by GDG Mien Trung\n\n11/2023\n\nDeputy Lead at GDSC-FPTU 10/2023 Deputy Lead at GDSC FPT University Da Nang\n\nGoogle Solution Challenge 06/2022 Solution Challenge competition by Google Developer Student Clubs\n\n07/04/2003\n\nObjective I am currently in the third year of FPT University and major is Artificial Intelli- gence. I am a code-holic guys and experimented with serveral Freelance jobs. I will building and developing AI solution with high accuracy and efficient.\n\nTechnical Skills\n\n8/10 Python Able to build apps, AI, libraries\n\nTensorFlow, PyTorch 5/10 Able to build custom ML, DL models\n\n7/10\n\nGit, Github Familiar with version control tools\n\n8/10\n\nFlask, FastAPI Able to build Back-end solution for AI\n\n3/10 Docker Able to create simple Container\n\nJira, Trello Familiar with task tracking tools\n\n8/10\n\nSoft Skills Team Working Able to work with medium team size\n\nLeadership Able to manage a team, efforts, performance, handle mental issues\n\nLanguages English Archived IELTS 5.5\n\nJapanese Language understanding
"""

In [15]:
cv3 = """
DIEP GIA DONG\n\nPERSONAL DETAILS\n\nName ● Nationality ● Date of Birth ● Sex ● Marital status ● Phone No. ● Email ● Github\n\nDIEP GIA DONG Vietnamese May, 29th 2003 Male Single (84) 356 16 91 37 diepgiadong@gmail.com github.com/DiepDong\n\nEDUCATIONAL BACKGROUND\n\nApplied Artificial intelligence - FPT Quy Nhon University, Vietnam\n\nPROFESSIONAL SUMMARY\n\nHard working, self-confidence, good communication skills. ● Work under pressure. ● High teamwork spirit and eager to learn and share. ● Sociable, friendly communication in team work.\n\nSOFTWARE\n\nDatabases etc: Qdrant.\n\nProgramming Languages: Python, HTML & CSS, Reactjs(basic).\n\nProgramming Tools: Visual Studio code, Docker, Kubernetes.\n\nOOAD/OOP: Object Oriented Programming (OOP).\n\nEDUCATIONAL EXPERIENCE\n\nPROJECTANDPRACTICE\n\nProjectname Position(s)\n\nSentiment classification Analysis and Front-end Developer\n\nDuration 6/2023 - 8/2023\n\nGeneral information\n\nCustomer: Vietnam\n\nTeam Size: 4 person\n\nDescription\n\nFrom input text emotion programs build AI models (output including other, Disgust, Enjoyment, Anger, Surprise, Sadness, and Fear).\n\nDiep Gia Dong's CV - Confidential\n\n1\n\nProject Scope\n\nTechnology used\n\nProjectname Position(s)\n\nGeneral information\n\nDescription\n\nProject Scope\n\nTechnology used\n\nRequirement - Design Web - Coding - Test\n\n▪ ▪ ▪\n\nPython Flask App HTML & CSS\n\nCoconut Mature Classification Pre-processing and Developer\n\nDuration 9/2023 - 12/2023\n\nCustomer: Vietnam\n\nTeam Size: 5 person\n\nFrom input coconut image programs build AI models (output including young coconut, mature coconut, old coconut)\n\nRequirement - Design Web - Coding - Test\n\n▪ ▪ ▪\n\nPython Flask APP HTML & CSS\n\nHOBBIES\n\nMusic, reading, travelling…\n\nDiep Gia Dong's CV - Confidential\n\n2
"""

In [18]:
# Fresher Web
cv4 = """
Name: Emily Chen
Email: emily.chen@email.com

Summary: Enthusiastic and dedicated Computer Science graduate from Georgia Tech with a strong interest in web technologies and user experience design. Excellent problem-solving skills and a keen learner of new technologies and frameworks.

Technical Skills: HTML, CSS, JavaScript, React.js, Python, MySQL, Basic PHP, Agile Methodologies

Experience:
- Developed a responsive portfolio website to showcase personal projects and academic research.
- Collaborated with peers to create a browser-based game using JavaScript and HTML5 Canvas.
- Volunteered to develop a website for a local non-profit, improving their event visibility and donations.

Education: Bachelor's in Computer Science, Georgia Institute of Technology

"""

In [22]:
# Junior Web
cv5 = """
Name: Jordan Taylor
Email: jordan.taylor@email.com

Summary: Passionate Software Developer with 3 years of experience in creating robust and user-centric web solutions. Adept in both front-end and back-end development with a track record of building scalable web applications. Committed to leveraging modern development practices and frameworks to achieve business goals and provide value to end-users.

Technical Skills: HTML5, CSS3, JavaScript (ES6+), React, Redux, Node.js, Express, MongoDB, SQL, RESTful API Development, Version Control (Git), Unit Testing (Jest)

Experience:
- Developed and launched a responsive SPA for an online retail store, leading to a 40% increase in customer retention.
- Collaborated in a cross-functional team to integrate RESTful services, improving data retrieval efficiency by 30%.
- Engineered a custom CMS using Node.js and MongoDB, which enhanced content management efficiency for non-technical staff.
- Fostered an Agile development environment, consistently meeting sprint goals and contributing to a 20% increase in overall development speed.

Education: Bachelor's in Software Engineering, University of California, Berkeley

"""

In [23]:
# Senior Web
cv6 = """
Name: Oliver Zhang
Email: oliver.zhang@email.com

Summary: Seasoned Full-Stack Developer with 8 years of experience in building complex web applications and leading agile development teams. Specializes in end-to-end development, from database design to front-end user interfaces, with a focus on creating efficient and scalable products.

Technical Skills: Full-Stack Development, React, Angular, .NET Core, Node.js, SQL Server, MongoDB, Elasticsearch, Azure Cloud Services, DevOps, Agile Scrum, Test-Driven Development

Experience:
- Led the development of a real-time analytics dashboard used by Fortune 500 companies, enhancing decision-making capabilities.
- Drove the migration of legacy systems to modern web frameworks, reducing operational costs by 40% and improving system uptime to 99.99%.
- Implemented DevOps practices across the organization, significantly reducing the release cycle time and improving system stability.

Education: Bachelor's in Computer Science, Massachusetts Institute of Technology (MIT)

"""

## Memory

In [24]:
memory = ConversationBufferMemory(memory_key="chat")

In [25]:
chat_history = memory.load_memory_variables({}).get("chat")
chat_history

''

In [ ]:
# memory.save_context({"input": "Who is the president of the USA"}, {"output": "Joe Biden"})

## Prompt + Context

In [ ]:
# def answer():
#     # Create the chat message, potentially incorporating previous insights
#     chat_history = memory.load_memory_variables({}).get("chat_history")
#     if chat_history:
#         chat_message = prompt.format_messages(context=chat_history)
#     else:
#         chat_message =  prompt.format_messages()

#     # Analyze with the chain
#     result = chain.invoke(chat_message)

#     # Update memory
#     memory.save_context({"inputs": chat_message[0].content}, {"output": result.content})

#     return result

In [ ]:
# chat_message = prompt.format_messages(context=chat_history)
# print(chat_message[0].content)

In [ ]:
# answer().content

## Chain

In [26]:
chain0 = LLMChain(llm=llm, prompt=prompt_jd, memory=memory)
chain0

LLMChain(memory=ConversationBufferMemory(memory_key='chat'), prompt=ChatPromptTemplate(input_variables=['jd'], messages=[SystemMessage(content='\n                Let\'s think step by step.\n                Act as a HR manager and analyze Job Description: {jd}\n                Analyzing guide:\n                Let\'s think step by step.\n                Respond using only the provided information and do not rely on your basic knowledge.\n                Only use the given data to determine educational qualifications and certificates; do not make assumptions about these qualifications.\n                However, you are allowed to combine the provided details to draw logical conclusions about soft skills.\n                Please structure the JSON output as follows: {"Job Description": "", "Requirements": ""}\n                Next, you will be given CVs. Analyze the CVs and determine if the candidates meet the job requirements. Be fair and objective in your assessment.\n                ')

In [27]:
chain1 = LLMChain(llm=llm, prompt=prompt_match, memory=memory)
chain1

LLMChain(memory=ConversationBufferMemory(memory_key='chat'), prompt=ChatPromptTemplate(input_variables=['cv'], messages=[SystemMessage(content='\n                I have a curriculum vitae (CV) for a candidate, and the above job description (JD) for a specific position. \n                This is the CV: \n                {cv}\n                Based on the provided CV and JD, I need an analysis that examines how well the candidate\'s qualifications match the job requirements. This analysis should be summarized in a JSON file, with special attention to:\n                - The match with the base requirements outlined in the JD, including education, technical skills, and relevant experiences.\n                - A detailed description of any additional qualifications the candidate possesses that align with the JD. This should focus on research experience, communication skills, and any other notable achievements, directly inserted into the \'bonus_qualifications_met\' field in the JSON.\n   

In [28]:
result0 = chain0.run(jd)
print(result0)

d:\FSA\fsa-simpleqt\backend-simpleqt\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{"Job Description": "Research and develop new AI solutions and technologies for the company's software projects using artificial intelligence technology.\nParticipate in knowledge sharing seminars and regular reports.\nUndertake other projects and tasks as required by the direct manager.", "Requirements": {"Education": "Graduate from a regular university majoring in Computer Science, Data Science, or Artificial Intelligence.", "Skills": ["Proficiency in one of the programming languages such as Python, C++, etc.", "Experience working with Git, Docker.", "Understand the frameworks: Tensorflow/keras, pytorch.", "Understand the architectures CNN, RNN, LSTM, Transformer.", "Understand the SSH network protocol.", "Read and understand English in the field of Information Technology.", "Good communication skills, ability to work independently and in a team."], "Experience": "Experience in participating in AI projects related to face processing systems is an advantage.", "Soft Skills": ["Be inte

In [34]:
result1 = chain1.run(cv6)
print(result1)

```json
{
  "match_with_base_requirements": "**Education:** The candidate holds a Bachelor's degree in Computer Science from MIT, which meets the JD's requirement for a bachelor's degree in a relevant field.\n\n**Technical Skills:** The candidate possesses a strong foundation in full-stack development, including proficiency in React, Angular, .NET Core, Node.js, SQL Server, MongoDB, Elasticsearch, Azure Cloud Services, DevOps, and Agile Scrum. This aligns well with the JD's requirement for expertise in full-stack development and familiarity with modern web frameworks and cloud services.\n\n**Relevant Experience:** The candidate's experience in leading the development of a real-time analytics dashboard, migrating legacy systems to modern web frameworks, and implementing DevOps practices demonstrates their ability to handle complex development projects and drive technological advancements. These experiences align with the JD's requirement for experience in full-stack development, system 

In [ ]:
memory.load_memory_variables({}).get("chat")

In [29]:
with open('chat_memory.pkl', 'wb') as f:
    pickle.dump(memory, f)

In [ ]:
with open(f'chat_memory.pkl', 'rb') as f:
    memory = pickle.load(f)